## 集約とグループ化
### 惑星データ
[Seabornライブラリ](http://seaborn.pydata.org)の一部として入手できるPlanetデータセットを使用する。<br>
このデータは天文学者が恒星の周辺で発見した惑星に関する情報を提供している。

In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


### pandasによる単純な集約
describe()メソッドは各列のさまざまな集約結果を返す。
ここでは、値の無い行は削除している。

In [4]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


In [5]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


pandasの集約メソッド  

|集約メソッド|説明|
|:------:|:------|
|count()|要素数|
|first(), last()|最初の要素、最後の要素|
|mean(), median()|平均値、中央値|
|min(), max()|最小値、最大値|
|std(), var()|標準偏差、分散|
|mad()|平均絶対偏差(Mean absolute deviation)|
|prod()|全要素の積|
|sum()|総計|

### GroupBy:分割、適用、結合
GroupByのステップ<br>
* 分割(split)：指定したキーの値に応じてDataFrameを分割してグループを作る
* 適用(apply)：それぞれのグループ内に対して、通常は集約、変換、またはフィルタリングなどを行う
* 結合(combine)：操作の結果を出力配列に結合する。

In [6]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                                  'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [7]:
df.groupby('key')

In [8]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


#### GroupByオブジェクト
GroupBy操作で実行できるその他の基本的な機能

In [9]:
# 列インデクス
planets.groupby('method')

In [11]:
planets.groupby('method')['orbital_period']

ここでは列名を指定して、元のDataFrameグループから特定のSeriesグループを抽出した。<br>
GroupByオブジェクトに対しては、集約操作を行うまで実際の計算は行われない。

In [12]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

それぞれの観測手法によるもっとも適した公転周期（日数）のおおよその大きさがわかる。

**グループごとの繰り返し**<br>
GroupByオブジェクトは、グループに対する反復をサポートし、各グループをSeriesまたはDataFrameとして返す。<br>
手動で特定の操作を行う場合には便利だが、apply機能を用いる方が高速

In [13]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


**メソッド呼び出し**<br>
GroupByオブジェクトより明示的に実装されていないメソッドは、DataFrameオブジェクトまたはSeriesオブジェクトのグループに対して呼び出される。<br>
例えば、DataFrameのdescribe()メソッドを使用して、データ内の各グループに対する一般的な集約操作を実行できる。<br>
DataFrameおよびSeriesに有効なメソッドはGroupByオブジェクトに対しても使用できる。

In [17]:
planets.groupby('method')['year'].describe()#.unstack()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


#### 集約、フィルタ、変換、適用
グループ化したデータを結合する前に様々な操作を効率的に実装するための以下のメソッドがある。<br>
* aggregate()：集計
* filter()：フィルタ
* transform()：変換
* apply：適用

In [20]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                                  'data1': range(6),
                                  'data2': rng.randint(0, 10, 6)},
                                 columns=['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


**集約**<br>
aggregate()メソッドは、文字列、関数、またはそのリストを取り、全ての集約を一度に計算することができる。

In [21]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [22]:
# 列名とその列に適用される操作のマッピングした辞書を渡す方法
df.groupby('key').aggregate({'data1': 'min',
                                                    'data2': 'max'})

,data2,data1
key,,
A,5,0
B,7,1
C,9,2


**フィルタ**<br>
フィルタ操作を行うと、データの性質に基づいてデータの削除が可能になる。<br>
例えば標準偏差がある臨界値よりも大きいグループを全て残すには次の操作を行う。

In [23]:
def filter_func(x):
    return x['data2'].std() > 4

print(df); print(df.groupby('key').std());
print(df.groupby('key').filter(filter_func))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


In [31]:
# ラムダ式を使った場合
print(df.groupby('key').filter(lambda x: x['data2'].std() > 4))

  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


**変換**<br>
変換はデータ全体の変換された値を作り、そこから再結合する。<br>
この変換では、入力と同じ形状の出力が得られる。<br>
各要素からグループごとの平均との差を取り、データのセンタリングを行うのがよく用いられる変換の例である。

In [28]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


**適用**<br>
apply()メソッドを使用すると、グループ化の結果に任意の関数を適用することができる。<br>
この関数はDataFrameを受け取り、pandasオブジェクト（例えばDataFram,Series）またはスカラーを返す必要がある。<br>
結合操作は返される出力の種類に合わせて調整される。<br>
例えば、最初の列を２番目の列の合計で正規化するapply()の例をいかに示す。

In [32]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

print(df); print(df.groupby('key').apply(norm_by_data2))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


#### 分割キーの指定

In [33]:
# リスト、配列、Series、Indexによるグループキーの指定
L = [0, 1, 0, 1, 2, 0]
print(df); print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
   data1  data2
0      7     17
1      4      3
2      4      7


In [34]:
# indexをグループにマップするSeriesおよび辞書
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C':'consonant'}
print(df2); print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
           data1  data2
consonant     12     19
vowel          3      8


In [35]:
# Python関数
# Index値を入力してグループのキーを出力する任意のPython関数を渡すことができる
print(df2); print(df2.groupby(str.lower).mean())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
   data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0


In [36]:
# キーのリスト
# 多重インデクスのグループ化が可能
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


In [42]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
